# Strands AgentsとOllamaモデルを使用したローカルエージェントの構築

このノートブックでは、Strands AgentとOllamaを使用してパーソナルエージェントを作成する方法をデモンストレーションします。このエージェントは、ファイル操作、ウェブ検索、システムコマンドなど様々なローカルタスクを実行できます。

## Ollamaとは？

[Ollama](https://ollama.com/)は、マシン上で大規模言語モデル（LLM）をローカル実行できるオープンソースフレームワークです。これらのモデルと対話するためのシンプルなAPIを提供し、外部サービスにデータを送信したくないプライバシー重視のアプリケーションに最適です。

Ollamaの主な利点：
- **プライバシー**: 全ての処理がローカルマシン上で実行
- **APIコストなし**: 好きなだけ無料で使用可能
- **オフライン対応**: インターネット接続不要で動作
- **カスタマイズ**: 特定の用途向けにファインチューニング可能


## エージェント詳細

<div style="float: left; margin-right: 20px;">
    
|機能                |説明                                              |
|--------------------|---------------------------------------------------|
|使用機能            |Ollamaモデル - ファイル操作エージェントの作成       |
|エージェント構造    |シングルエージェントアーキテクチャ                  |


</div>


### エージェントアーキテクチャ

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

In [ ]:
!pip install -r requirements.txt

## セットアップとインストール

このノートブックを実行する前に、以下の準備をしてください：

1. Ollamaのインストール: [https://ollama.com/download](https://ollama.com/download)の指示に従う
2. Ollamaサーバーの起動: `ollama serve`
3. Ollamaでモデルのダウンロード: `ollama pull llama3.2:1b`

詳細な指示は[ドキュメント](https://cuddly-sniffle-lrmk2y7.pages.github.io/0.1.x/user-guide/concepts/model-providers/ollama/)を参照してください。

このノートブックでは、SageMaker Studioとの互換性のためにLinuxディストリビューション用のOllamaをダウンロードします。これはAWSリードワークショップでWorkshop Studio上でコード実行するためのものです。ローカルでこのコードを実行する場合は、現在の環境に合わせてOllamaのダウンロード手順を調整してください。

In [ ]:
# this will work on linux computers
!curl -fsSL [https://ollama.com/install.sh](https://ollama.com/install.sh) | sh

In [ ]:
import subprocess
subprocess.Popen(['ollama', 'serve'])

In [ ]:
!ollama pull llama3.2:3b

In [ ]:
# Import necessary libraries
import os

import requests

# Import strands components
from strands import Agent, tool
from strands.models.ollama import OllamaModel

In [ ]:
# Check if Ollama is running:
try:
    response = requests.get("http://localhost:11434/api/tags")
    print("✅ Ollama is running. Available models:")
    for model in response.json().get("models", []):
        print(f"- {model['name']}")
except requests.exceptions.ConnectionError:
    print("❌ Ollama is not running. Please start Ollama before proceeding.")

## カスタムツールの定義

ツールは、エージェントが環境と対話するための関数です。以下にパーソナルエージェントに役立ついくつかのツールを定義します。

In [ ]:
# File Operation Tools
@tool
def file_read(file_path: str) -> str:
    """Read a file and return its content. Supports both text and PDF files.

    Args:
        file_path (str): Path to the file to read

    Returns:
        str: Content of the file

    Raises:
        FileNotFoundError: If the file doesn't exist
    """
    try:
        # Check if it's a PDF file
        if file_path.lower().endswith('.pdf'):
            import PyPDF2
            with open(file_path, "rb") as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text if text.strip() else "Error: Could not extract text from PDF"
        else:
            # Regular text file
            with open(file_path, "r", encoding="utf-8") as file:
                return file.read()
    except FileNotFoundError:
        return f"Error: File '{file_path}' not found."
    except Exception as e:
        return f"Error reading file: {str(e)}"


@tool
def file_write(file_path: str, content: str) -> str:
    """Write content to a file.

    Args:
        file_path (str): The path to the file
        content (str): The content to write to the file

    Returns:
        str: A message indicating success or failure
    """
    try:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(os.path.abspath(file_path)), exist_ok=True)

        with open(file_path, "w") as file:
            file.write(content)
        return f"File '{file_path}' written successfully."
    except Exception as e:
        return f"Error writing to file: {str(e)}"


@tool
def list_directory(directory_path: str = ".") -> str:
    """List files and directories in the specified path.

    Args:
        directory_path (str): Path to the directory to list

    Returns:
        str: A formatted string listing all files and directories
    """
    try:
        items = os.listdir(directory_path)
        files = []
        directories = []

        for item in items:
            full_path = os.path.join(directory_path, item)
            if os.path.isdir(full_path):
                directories.append(f"Folder: {item}/")
            else:
                files.append(f"File: {item}")

        result = f"Contents of {os.path.abspath(directory_path)}:\n"
        result += (
            "\nDirectories:\n" + "\n".join(sorted(directories))
            if directories
            else "\nNo directories found."
        )
        result += (
            "\n\nFiles:\n" + "\n".join(sorted(files)) if files else "\nNo files found."
        )

        return result
    except Exception as e:
        return f"Error listing directory: {str(e)}"

## Ollama駆動エージェントの作成

これでOllamaモデルと上記のツールを使用してエージェントを作成します。

注意: `execute_commands`、`search_files`などのツールを追加することもできます。

In [ ]:
# Define a comprehensive system prompt for our agent
system_prompt = """
You are a helpful personal assistant capable of performing local file actions and simple tasks for the user.

Your key capabilities:
1. Read, understand, and summarize files.
2. Create and write to files.
3. List directory contents and provide information on the files.
4. Summarize text content

When using tools:
- Always verify file paths before operations
- Be careful with system commands
- Provide clear explanations of what you're doing
- If a task cannot be completed, explain why and suggest alternatives

Always be helpful, concise, and focus on addressing the user's needs efficiently.
"""

model_id = (
    "llama3.2:3b"  # You can change this to any model you have pulled with Ollama.
)

#### Ollamaモデルの設定
Ollamaサービスが http://localhost:11434 で動作しており、`model_id`が上記に表示されたOllamaモデルのリストに含まれていることを確認してください。

In [ ]:
ollama_model = OllamaModel(
    model_id=model_id,
    host="http://localhost:11434",
    max_tokens=4096,  # Adjust based on your model's capabilities
    temperature=0.7,  # Lower for more deterministic responses, higher for more creative
    top_p=0.9,  # Nucleus sampling parameter
)

# Create the agent
local_agent = Agent(
    system_prompt=system_prompt,
    model=ollama_model,
    tools=[file_read, file_write, list_directory],
)

## エージェントのテスト

いくつかの例タスクでエージェントをテストしてみましょう。

In [ ]:
local_agent(
    "Read the file in the path `sample_file/Amazon-com-Inc-2023-Shareholder-Letter.pdf` and summarize it in 5 bullet points."
)

In [ ]:
# Example 2: List files in the current directory
response = local_agent("Show me the files in the current directory")

In [ ]:
# Example 3: Create a sample file
response = local_agent(
    "Create a file called 'sample.txt' with the content 'This is a test file created by my Ollama agent.'"
)

In [ ]:
# Example 4: create a readme file after reading and understanding multiple files
response = local_agent("Create a readme.md for the current directory")

## 結論

このノートブックで、StandsとOllamaを使用してローカルパーソナルエージェントを作成しました。このエージェントはファイル操作（読み書き、追記）とテキストの要約/分析ができます。

これにより、OllamaでAIモデルをローカル実行し、strandsのツールシステムの柔軟性を組み合わせた強力さが実証されました。必要に応じてさらに多くのツールを追加したり、異なるOllamaモデルを使用できます。

### 次のステップ（アイデア）

- エージェントの能力に影響を与える異なるOllamaモデルを試す
- ウェブ検索、メール送信、カレンダー統合などの複雑なツールを追加
- エージェントに過去の対話を記憶するメモリを実装
- エージェントとの対話のためのシンプルなUIを作成